# Mini-Project: 한영 번역기 만들기

> 4.5 장에 해당하는 코드

## 패키지 로딩

In [1]:
# 코드 4-4

from encdec import EncoderDecoder
from torchtext.vocab import Vectors
from torchtext.data import Field, TabularDataset, Iterator
from konlpy.tag import Mecab
from gensim.models.word2vec import Word2Vec
import spacy
import torch
import torch.nn as nn
import torch.optim as optim

## Word2Vec 알고리즘으로 토큰 벡터화

In [2]:
# 코드 4-5

# 영어 토큰화 함수로 SpaCy 사용
spacy_en = spacy.load('en')

# 한글 토큰화 함수로 MeCab 사용
mecab = Mecab()

# 각 언어에 해당하는 tokenizer 함수를 정의 한다.
tokenizer_ko = lambda x: [tkn.lower() for tkn in Mecab().morphs(x)]
tokenizer_en = lambda x: [tkn.text.lower() for tkn in spacy_en.tokenizer(x)]

with open("./data/eng-kor.tsv") as file:
    # 데이터를 불러와서 tab 을 기준으로 분리한다.
    raw_data = file.read().splitlines()
    eng, kor = zip(*[line.split("\t") for line in raw_data])
    # 토큰화를 진행한다.
    eng = [tokenizer_en(sent) for sent in eng]
    kor = [tokenizer_ko(sent) for sent in kor]
    

# 영어 Word2Vec 훈련
model_eng = Word2Vec(sentences=eng, size=100, window=3, min_count=1, sg=1)
model_eng.init_sims(replace=True)  # 훈련 완료후 불필요한 메모리 제거
model_eng.wv.save_word2vec_format("./word2vec_eng.pt")  # 모델 저장    

# 한글 Word2Vec 훈련
model_kor = Word2Vec(sentences=kor, size=100, window=3, min_count=1, sg=1)
model_kor.init_sims(replace=True)  # 훈련 완료후 불필요한 메모리 제거
model_kor.wv.save_word2vec_format("./word2vec_kor.pt")  # 모델 저장

## 필드정의 및 데이터 전처리

In [3]:
# 코드 4-6

# 필드 정의
SRC = Field(sequential=True,
            use_vocab=True,
            tokenize=tokenizer_en,  
            lower=True, 
            batch_first=True)  
TRG = Field(sequential=True,  
            use_vocab=True, 
            tokenize=tokenizer_ko,
            init_token="<s>",
            eos_token="</s>",
            batch_first=True)


# TabularDataset를 사용해 훈련 세트를 만든다.
train_data = TabularDataset(path="./data/eng-kor.tsv", format='tsv',
                            fields=[('eng', SRC), ('kor', TRG)])

# 학습한 Word2Vector를 Vectors 객체를 통해 불러온다.
vectors_kor = Vectors(name="./word2vec_kor.pt")
vectors_eng = Vectors(name="./word2vec_eng.pt")

# 단어장 생성
SRC.build_vocab(train_data.eng, min_freq=1, vectors=vectors_eng)
TRG.build_vocab(train_data.kor, min_freq=1, vectors=vectors_kor)
# 데이터 개수 및 단어장 크기 확인
print("Source Vocab Size: {}".format(len(SRC.vocab)))
print("Target Vocab Size: {}".format(len(TRG.vocab)))

# 환경 변수 설정
BATCH = 32  # 미니배치크기
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'  # 디바이스
STEP = 200  # 총 반복스텝

# 데이터 로더 생성
train_loader = Iterator(dataset=train_data, batch_size=BATCH, device=DEVICE)

Source Vocab Size: 1173
Target Vocab Size: 1549


In [4]:
idx = torch.randint(0, len(train_data), (1,))

print(train_data.examples[idx].eng)
print(train_data.examples[idx].kor)

['have', 'you', 'eaten', 'dinner', 'yet', '?']
['아직', '저녁', '을', '안', '드셨', '나요', '?']


## 모델 선언

In [5]:
# 코드 4-7

# 모델 선언에 필요한 인자 설정
enc_vocab_size = len(SRC.vocab)
dec_vocab_size = len(TRG.vocab)  
embed_size = 100  # E: 임베딩 크기
hidden_size = 600  # D: 은닉층 크기
num_layers = 2  # RNN 층의 개수
batch_first = True  # RNN 입력의 첫번째 차원이 미니배치 크기인 경우 활성화
bidirec = True  # Encoder의 양방향 순환 신경망 사용 여부

# 모델 선언
model = EncoderDecoder(
    enc_vocab_size=enc_vocab_size, 
    dec_vocab_size=dec_vocab_size, 
    embed_size=embed_size, 
    hidden_size=hidden_size, 
    num_layers=num_layers, 
    batch_first=True, 
    bidirec=bidirec, 
    sos_idx=TRG.vocab.stoi.get("<s>")).to(DEVICE)

# 각 encoder 와 decoder 에서 embedding 층을 미리 학습된 벡터로 교체해준다.
model.encoder.embedding = nn.Embedding.from_pretrained(SRC.vocab.vectors, freeze=False).to(DEVICE)
model.decoder.embedding = nn.Embedding.from_pretrained(TRG.vocab.vectors, freeze=False).to(DEVICE)

# 손실함수와 옵티마이저 선언
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())
# StepLR 스케쥴러는 Learing Rate 를 조절 해주는 함수
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.5)

## 모델 훈련 및 테스트

In [6]:
# 코드 4-8

best_loss = 50
best_step = None 
for step in range(STEP):
    scheduler.step()
    total_loss = 0
    for batch in train_loader:
        src_data, trg_data = batch.eng, batch.kor
        # 경사 초기화
        optimizer.zero_grad()
        # 순방향전파
        scores = model(src_data, maxlen=trg_data.size(1))
        # 손실값 계산
        train_loss = loss_function(scores.view(-1, scores.size(-1)), 
                                   trg_data[:, 1:].contiguous().view(-1))
        
        # 역방향 전파
        train_loss.backward()
        # 매개변수 업데이트
        optimizer.step()
        # 기록용
        total_loss += train_loss.item()
    
    # 모델 저장 여부 판단
    if total_loss <= best_loss:
        torch.save(model.state_dict(), "./encdec.pt")
        best_loss = total_loss
        best_step = step
        
    # 중간 과정 print    
    if step % 10 == 0:
        print("Step [{}] Loss: {:.4f}".format(step+1, total_loss))
        if best_step is not None:
            print("Model Saved at step: {}, best_loss: {:.4f}".format(best_step+1, best_loss))

Step [1] Loss: 148.8715
Step [11] Loss: 89.9731
Step [21] Loss: 43.4289
Model Saved at step: 21, best_loss: 43.4289
Step [31] Loss: 19.7057
Model Saved at step: 31, best_loss: 19.7057
Step [41] Loss: 25.5950
Model Saved at step: 36, best_loss: 14.8238
Step [51] Loss: 5.8047
Model Saved at step: 51, best_loss: 5.8047
Step [61] Loss: 3.1677
Model Saved at step: 61, best_loss: 3.1677
Step [71] Loss: 4.2973
Model Saved at step: 62, best_loss: 3.1487
Step [81] Loss: 4.6207
Model Saved at step: 62, best_loss: 3.1487
Step [91] Loss: 4.0227
Model Saved at step: 62, best_loss: 3.1487
Step [101] Loss: 2.9694
Model Saved at step: 101, best_loss: 2.9694
Step [111] Loss: 2.3598
Model Saved at step: 111, best_loss: 2.3598
Step [121] Loss: 2.2942
Model Saved at step: 121, best_loss: 2.2942
Step [131] Loss: 2.3475
Model Saved at step: 126, best_loss: 2.2230
Step [141] Loss: 2.5317
Model Saved at step: 126, best_loss: 2.2230
Step [151] Loss: 2.2445
Model Saved at step: 126, best_loss: 2.2230
Step [161]

### 테스트

In [11]:
# 코드 4-9

model.eval()
test_sent = "I'm shocked.".lower()
# 테스트할 문장을 텐서로 변환한다.
src_tensor = SRC.process([tokenizer_en(test_sent)]).to(DEVICE)
# Encoder 로 소스 문장을 압축한다.
enc_outputs = model.encoder(src_tensor)
# Decoder 로 타겟 문장을 예측한다.
scores = model.decoder(enc_outputs, maxlen=50, eos_idx=TRG.vocab.stoi["</s>"])
pred = scores.argmax(-1).squeeze()[:-1].cpu().tolist()
decode = lambda li: [TRG.vocab.itos[idx] for idx in li] 
# 예측
print(test_sent)
print(" ".join(decode(pred)))

i'm shocked.
충격 이 야 .
